In [1]:
import pandas as pd
import numpy as np
import getschools as gs

In [106]:
DATA_YEAR="2014-15"

def get_state(level="elementary",skiprows=3):
    
    ret = pd.read_excel("../raw/class size 6444.xlsx",sheetname="STATEWIDE_" + level,skiprows=skiprows)
    ret["year"] = DATA_YEAR
    a
    ret = gs.clean_colnames(ret)
    return ret

elem_state = get_state(level="elementary")
elem_state

,grade,n_obs,n,mean,year
0,1,5150,5150,19.539417,2014-15
1,2,5089,5089,20.556101,2014-15
2,3,4804,4804,20.932764,2014-15
3,4,5070,5070,21.394477,2014-15
4,5,5597,5597,20.907808,2014-15
5,6,7941,7941,20.549049,2014-15
6,7,7910,7910,20.289760,2014-15
7,8,7959,7959,20.377937,2014-15
8,KG,4651,4651,19.138895,2014-15
9,PK,1262,1262,16.095087,2014-15


In [78]:
#get_state(level="Secondary",skiprows=4)
#get_state(level="Secondary",skiprows=4)
state_dfs = {
    "Secondary":get_state(level="Secondary",skiprows=4),
    "elementary:":get_state(level="elementary")
}
state_dfs["Secondary"]

,ncescoursename,n,mean,year
0,AP Biology,212,16.561321,2014-15
1,AP Calculus AB,206,17.121359,2014-15
2,AP Calculus BC,73,16.849315,2014-15
3,AP Chemistry,140,15.764286,2014-15
4,AP Comparative Government and Politics,17,16.647059,2014-15
5,AP Economics,38,21.447368,2014-15
6,AP English Language and Composition,310,18.712903,2014-15
7,AP English Literature and Composition,272,17.224265,2014-15
8,AP Environmental Science,110,17.063636,2014-15
9,AP European History,82,16.060976,2014-15


In [41]:
# get the weighted average of high school class sizes
def get_state_weighted_avg_hs():
    df = get_state(level="Secondary",skiprows=4)
    
    
    ret = [{
            "year":DATA_YEAR,
            "level":"statewide",
            "avg_class_size":np.average(df["mean"],weights=df["n"]),
    }]
    
    return pd.DataFrame(ret)

get_state_weighted_avg_hs()

,avg_class_size,level,year
0,19.021926,statewide,2014-15


# School-level 

In [54]:
def clean_code(n,fill_len):
    
    try:
        return str(int(n)).zfill(fill_len)
    except:
        return
    
    

In [63]:
def w_avg(df, mean_col, w_col):
    return np.average(df[mean_col],weights=df[w_col])

In [64]:
def facility_df(df, school_name, school_col="school"):
    return df[df[school_col] == school_name]

In [71]:
def facility_weighted_avg(df, school_name, school_col="school", mean_col="mean",w_col="n"):
    ret = facility_df(df, school_name, school_col)
    
    #print ret[school_col].unique()

    return w_avg(ret, mean_col,w_col)

In [130]:
def get_school(level="elementary"):
    
    ret = pd.read_excel("../raw/class size 6444.xlsx",
                        sheetname="Facility_" + level, 
                        dtypes={"Facility1Code":str,
                              "District Code":str},
                        skiprows=4)
    
        
    ret = gs.clean_colnames(ret)
    
    ret["district_name"] = ret["district_name"].fillna(method="ffill")
    ret["school"] = ret["facility_name"].fillna(method="ffill")

    ret["district_code"] = ret["district_code"].apply(lambda x: clean_code(x, 3)).fillna(method="ffill")
    ret["school_code"] = ret["facility1code"].apply(lambda x: clean_code(x, 7)).fillna(method="ffill")
    
    ret = gs.add_district_code(ret)
    #ret["district_code"] = ret["dist_code"]
    ret["level"] = level.title()
    ret["year"] = DATA_YEAR
    
    ret["w_avg"] = ret.apply(lambda x: facility_weighted_avg(ret, x["school"]),axis=1)
    
    return ret

high_school_detail_df = get_school(level="Secondary")
high_school_detail_df.sort_values(by="w_avg",ascending=False)
    

,district_code,district_name,facility1code,facility_name,ncescoursename,n,mean,school,school_code,level,year,w_avg
7301,2820013,Stamford Academy District,NaN,NaN,Integrated Science,1,41.0,Stamford Academy,2826013,Secondary,2014-15,39.736842
7298,2820013,Stamford Academy District,NaN,NaN,English/Language Arts III (11th grade),1,51.0,Stamford Academy,2826013,Secondary,2014-15,39.736842
7292,2820013,Stamford Academy District,NaN,NaN,Biology,1,35.0,Stamford Academy,2826013,Secondary,2014-15,39.736842
7293,2820013,Stamford Academy District,NaN,NaN,Chemistry,1,51.0,Stamford Academy,2826013,Secondary,2014-15,39.736842
7294,2820013,Stamford Academy District,NaN,NaN,Civics,1,39.0,Stamford Academy,2826013,Secondary,2014-15,39.736842
7295,2820013,Stamford Academy District,NaN,NaN,English Language and Literature - Independent ...,1,12.0,Stamford Academy,2826013,Secondary,2014-15,39.736842
7296,2820013,Stamford Academy District,NaN,NaN,English/Language Arts I (9th grade),1,41.0,Stamford Academy,2826013,Secondary,2014-15,39.736842
7297,2820013,Stamford Academy District,NaN,NaN,English/Language Arts II (10th grade),1,36.0,Stamford Academy,2826013,Secondary,2014-15,39.736842
7299,2820013,Stamford Academy District,NaN,NaN,English/Language Arts IV (12th grade),1,37.0,Stamford Academy,2826013,Secondary,2014-15,39.736842
7290,2820013,Stamford Academy District,2826013.0,Stamford Academy,Algebra - Other,1,32.0,Stamford Academy,2826013,Secondary,2014-15,39.736842


In [131]:
elem_school_detail_df = get_school(level="elementary")
elem_school_detail_df

,district_code,district_name,facility1code,facility_name,grade,n,mean,school,school_code,level,year,w_avg
0,0010011,Andover School District,10111.0,Andover Elementary School,1,2,15.000000,Andover Elementary School,0010111,Elementary,2014-15,16.235294
1,0010011,Andover School District,NaN,NaN,2,3,13.333333,Andover Elementary School,0010111,Elementary,2014-15,16.235294
2,0010011,Andover School District,NaN,NaN,3,2,17.500000,Andover Elementary School,0010111,Elementary,2014-15,16.235294
3,0010011,Andover School District,NaN,NaN,4,2,15.500000,Andover Elementary School,0010111,Elementary,2014-15,16.235294
4,0010011,Andover School District,NaN,NaN,5,3,15.333333,Andover Elementary School,0010111,Elementary,2014-15,16.235294
5,0010011,Andover School District,NaN,NaN,6,2,20.500000,Andover Elementary School,0010111,Elementary,2014-15,16.235294
6,0010011,Andover School District,NaN,NaN,KG,2,12.000000,Andover Elementary School,0010111,Elementary,2014-15,16.235294
7,0010011,Andover School District,NaN,NaN,PK,1,29.000000,Andover Elementary School,0010111,Elementary,2014-15,16.235294
8,0020011,Ansonia School District,20311.0,Mead School,1,4,20.500000,Mead School,0020311,Elementary,2014-15,23.678571
9,0020011,Ansonia School District,NaN,NaN,2,4,22.000000,Mead School,0020311,Elementary,2014-15,23.678571


In [132]:
subj_cache = {}

def get_state_subject_mean(subject,level="Secondary"):
    
    if level not in subj_cache:
        subj_cache[level] = {}
        
    if subject in subj_cache[level]:
        return subj_cache[level][subject]
        
    df = state_dfs[level]
    
    matches = df[df["ncescoursename"].str.upper() == subject.upper()]
    
    if len(matches) != 1:
        print "Error: Match count (" + str(len(matches)) + ") for '" + subject + "'"
        return 
    
    ret = float(matches["mean"]),int(matches["n"])
    
    if subject not in subj_cache[level]:
        subj_cache[level][subject] = ret
    #print matches
    
    return ret
    
    
get_state_subject_mean("AP Biology")

(16.5613208, 212)

In [133]:
def hs_add_state_subject_mean(df):
    
    level = "Secondary"
    ret = df.copy()
    def add_st_mean_n(row):
        #print row
        return get_state_subject_mean(row["ncescoursename"],level=level)
        
    ret["state_mean"] = ret["ncescoursename"].apply(lambda x: get_state_subject_mean(x,level=level)[0])
    ret["state_n"] = ret["ncescoursename"].apply(lambda x: get_state_subject_mean(x,level=level)[1])
    
    return ret
    
        
hs = hs_add_state_subject_mean(high_school_detail_df)
hs.head()

,district_code,district_name,facility1code,facility_name,ncescoursename,n,mean,school,school_code,level,year,w_avg,state_mean,state_n
0,0020011,Ansonia School District,20111.0,P.A.C.E. (Positive And Creative Education),Algebra II,1,7.0,P.A.C.E. (Positive And Creative Education),0020111,Secondary,2014-15,7.928571,20.364433,1597
1,0020011,Ansonia School District,NaN,NaN,Biology - Other,2,9.0,P.A.C.E. (Positive And Creative Education),0020111,Secondary,2014-15,7.928571,17.236111,72
2,0020011,Ansonia School District,NaN,NaN,Civics,1,8.0,P.A.C.E. (Positive And Creative Education),0020111,Secondary,2014-15,7.928571,19.912409,822
3,0020011,Ansonia School District,NaN,NaN,Contemporary U.S. Issues,1,7.0,P.A.C.E. (Positive And Creative Education),0020111,Secondary,2014-15,7.928571,15.566038,53
4,0020011,Ansonia School District,NaN,NaN,English/Composition (juniors and seniors),1,7.0,P.A.C.E. (Positive And Creative Education),0020111,Secondary,2014-15,7.928571,19.425287,87


In [134]:
def es_get_grade_mean(grade):
    matches = elem_state[elem_state["grade"] == grade]
    
    if len(matches) != 1:
        print "Error: " + str(len(matches)) + " matches found for grade " + str(grade)
        return
    
    return float(matches["mean"]), float(matches["n"])
    
es_get_grade_mean(1)   
es_get_grade_mean("PK")

(16.0950872, 1262.0)

In [135]:
def es_add_state_grad_mean(df):
    
    ret = df.copy()
    ret["state_mean"] = ret["grade"].apply(lambda x:es_get_grade_mean(x)[0])
    ret["state_n"] = ret["grade"].apply(lambda x:es_get_grade_mean(x)[1])

    return ret

es = es_add_state_grad_mean(elem_school_detail_df)
es.head()

,district_code,district_name,facility1code,facility_name,grade,n,mean,school,school_code,level,year,w_avg,state_mean,state_n
0,0010011,Andover School District,10111.0,Andover Elementary School,1,2,15.000000,Andover Elementary School,0010111,Elementary,2014-15,16.235294,19.539417,5150.0
1,0010011,Andover School District,NaN,NaN,2,3,13.333333,Andover Elementary School,0010111,Elementary,2014-15,16.235294,20.556101,5089.0
2,0010011,Andover School District,NaN,NaN,3,2,17.500000,Andover Elementary School,0010111,Elementary,2014-15,16.235294,20.932764,4804.0
3,0010011,Andover School District,NaN,NaN,4,2,15.500000,Andover Elementary School,0010111,Elementary,2014-15,16.235294,21.394477,5070.0
4,0010011,Andover School District,NaN,NaN,5,3,15.333333,Andover Elementary School,0010111,Elementary,2014-15,16.235294,20.907808,5597.0


In [136]:
print len(es[es["school_code"].isnull()])
print len(es[es["school_code"].notnull()])

0
4488


In [137]:
print len(hs[hs["school_code"].isnull()])
print len(hs[hs["school_code"].notnull()])

0
7971


In [150]:
def unify():
    global es, hs
    
    #print "cleaning es"
    #print es.columns
    
    es_copy = es[["district_code","district_name","school_code","school","grade","level","mean","n","state_mean","state_n","year"]]
    es_copy.columns = ["district_code","district_name","school_code","school","subject",
                       "school_level","mean","n","state_mean","state_n","year"]
    
    #print "cleaning hs"
    #print hs.columns

    hs_copy = hs[["district_code","district_name","school_code","school","ncescoursename","level","mean","n","state_mean","state_n","year"]]
    hs_copy.columns = ["district_code","district_name","school_code","school","subject",
                       "school_level","mean","n","state_mean","state_n","year"]
    
    ret = pd.concat([es_copy,hs_copy])
    ret["mean"] = ret["mean"].apply(lambda x: round(x)).astype(int)
    ret["state_mean"] = ret["state_mean"].apply(lambda x: round(x)).astype(int)
    
    #ret = ret.sort_values(by="subject")
    return ret

mf = unify()
mf[mf["school"] == "Bethel High School"]

,district_code,district_name,school_code,school,subject,school_level,mean,n,state_mean,state_n,year
159,0090011,Bethel School District,0096111,Bethel High School,AP Biology,Secondary,15,1,17,212.0,2014-15
160,0090011,Bethel School District,0096111,Bethel High School,AP Calculus AB,Secondary,21,2,17,206.0,2014-15
161,0090011,Bethel School District,0096111,Bethel High School,AP Calculus BC,Secondary,16,1,17,73.0,2014-15
162,0090011,Bethel School District,0096111,Bethel High School,AP English Language and Composition,Secondary,25,1,19,310.0,2014-15
163,0090011,Bethel School District,0096111,Bethel High School,AP English Literature and Composition,Secondary,9,1,17,272.0,2014-15
164,0090011,Bethel School District,0096111,Bethel High School,AP Environmental Science,Secondary,14,1,17,110.0,2014-15
165,0090011,Bethel School District,0096111,Bethel High School,AP European History,Secondary,10,1,16,82.0,2014-15
166,0090011,Bethel School District,0096111,Bethel High School,AP Human Geography,Secondary,20,2,19,28.0,2014-15
167,0090011,Bethel School District,0096111,Bethel High School,AP Physics B,Secondary,17,3,19,143.0,2014-15
168,0090011,Bethel School District,0096111,Bethel High School,AP Psychology,Secondary,20,3,21,275.0,2014-15


In [144]:
mf.to_csv("../clean/for_db/class_size.csv", index=False)